In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
from tsai.all import *
import sklearn.metrics as skm
import sys
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os

sys.path.append('../pre_processing_and_recover_network')
import data_preprocessing
my_setup()

In [ ]:
# prepare the data
def split_data_indexes(len_data, train_ratio):
    indexes = list(range(len_data))
    random.shuffle(indexes)
    train_size = int(len_data * train_ratio)
    train_indexes = indexes[:train_size]
    test_indexes = indexes[train_size:]
    return (train_indexes, test_indexes)

swell_data = data_preprocessing.load_data(os.path.join("../pre_processing_and_recover_network/swell_dataset/","swell_dict.npy"))
swell_data = data_preprocessing.swell_prepare_for_10fold(swell_data)
swell_data[:, 1:4] -= 1
x = swell_data[:, 4:]

y_arousal = swell_data[:, 2]
y_arousal = y_arousal.reshape(y_arousal.shape[0],)

y_valence = swell_data[:, 3]
y_valence = y_valence.reshape(y_valence.shape[0],)

y_stress = swell_data[:, 1]
y_stress = y_stress.reshape(y_stress.shape[0],)

print(x.shape)
print(type(x))
print(y_stress.shape)
print(y_valence.shape)
print(y_arousal.shape)


# set X to 3-Dim array
x = np.array(x)
x = np.expand_dims(x, axis=1)
print(x.shape)
filename = 'X_memmap.npy'
X = np.memmap(filename, dtype='float32', mode='w+', shape=x.shape)
X[:] = x[:]
print(X.shape)
print(type(X))

filename = 'Y_arousal_memmap.npy'
Y_arousal = np.memmap(filename, dtype='float32', mode='w+', shape=y_arousal.shape)
Y_arousal[:] = y_arousal[:]

filename = 'Y_valence_memmap.npy'
Y_valence = np.memmap(filename, dtype='float32', mode='w+', shape=y_valence.shape)
Y_valence[:] = y_valence[:]

filename = 'Y_stress_memmap.npy'
Y_stress = np.memmap(filename, dtype='float32', mode='w+', shape=y_stress.shape)
Y_stress[:] = y_stress[:]
print(Y_stress.shape)
print(type(Y_stress))
splits=split_data_indexes(X.shape[0],0.9)
print(len(splits[0]))
print(len(splits[1]))

**For randomal splits**

In [ ]:
splits=np.load('./splits_A.npy',allow_pickle=True)

**For spliting subjests for Train and Test**

In [ ]:
splits=np.load('./splits_B.npy',allow_pickle=True)

**Prepare dataset loaders and model**

In [3]:
tfms  = [None, [Categorize()]]
splits=(splits[0],splits[1])
dsets_arousal = TSDatasets(X, Y_arousal, tfms=tfms, splits=splits, inplace=True)
dsets_valence = TSDatasets(X, Y_valence, tfms=tfms, splits=splits, inplace=True)
dsets_stress = TSDatasets(X, Y_stress, tfms=tfms, splits=splits, inplace=True)

bs=128
dls_stress = TSDataLoaders.from_dsets(dsets_stress.train, dsets_stress.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_arousal = TSDataLoaders.from_dsets(dsets_arousal.train, dsets_arousal.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_valence = TSDataLoaders.from_dsets(dsets_valence.train, dsets_valence.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)

model_stress = InceptionTime(dls_stress.vars, dls_stress.c)
model_arousal = InceptionTime(dls_arousal.vars, dls_arousal.c)
model_valence = InceptionTime(dls_valence.vars, dls_valence.c)

learn_stress = Learner(dls_stress, model_stress, metrics=accuracy)
learn_arousal = Learner(dls_arousal, model_arousal, metrics=accuracy)
learn_valence = Learner(dls_valence, model_valence, metrics=accuracy)

learn_stress.save('S0')
learn_arousal.save('S0')
learn_valence.save('S0')

Path('models/S0.pth')

In [ ]:
print(model_stress)

**Train for stress**

In [6]:
learn_stress.load('S0')
learn_stress.lr_find()

SuggestedLRs(valley=0.0020892962347716093)

In [ ]:
learn_stress.fit_one_cycle(100,lr_max=1e-3)
learn_stress.save('S1')

In [8]:
learn_stress.save_all(path='./stress_inc', dls_fname='dls_stress', model_fname='model_stress', learner_fname='learner_stress')

**Train for arousal**

In [9]:
learn_arousal.load('S0')
learn_arousal.lr_find()

SuggestedLRs(valley=0.0003311311302240938)

In [ ]:
learn_arousal.fit_one_cycle(100,lr_max=1e-4)
learn_arousal.save('S1')

In [11]:
learn_arousal.save_all(path='./arousal_inc', dls_fname='dls_arousal', model_fname='model_arousal', learner_fname='learner_arousal')

**Train for valence**

In [12]:
learn_valence.load('S0')
learn_valence.lr_find()

SuggestedLRs(valley=0.001737800776027143)

In [ ]:
learn_valence.fit_one_cycle(100,lr_max=1e-3)
learn_valence.save('S1')

In [14]:
learn_valence.save_all(path='./valence_inc', dls_fname='dls_valence', model_fname='model_valence', learner_fname='learner_valence')

**Splits subjests to train and test-locally**

In [4]:
def train_test_indexes_var2(ratio, objects):
    # Split objects into train and test sets
    num_test = int(len(set(objects)))-int(len(set(objects)) * ratio)
    obj_list = list(set(objects))
    random.shuffle(obj_list)
    test_objs = obj_list[:num_test]
    train_objs = obj_list[num_test:]

    train_idxs = []
    test_idxs = []

    # Assign each object's samples to the appropriate set
    for obj in train_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        train_idxs.extend(obj_idxs)

    for obj in test_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        test_idxs.extend(obj_idxs)

    # Shuffle the final train and test sets
    random.shuffle(train_idxs)
    random.shuffle(test_idxs)

    return (train_idxs), (test_idxs)

In [ ]:
# check train_test_indexes_var2()
# not for each runnig acieve exaclty 9:1 train:test
ratio = 0.9
objects=swell_data[:,0]
print("tot len is ", len(objects))
print("trin len is ",ratio*len(objects))
print("test len is ",len(objects)-ratio*len(objects))
i=train_test_indexes_var2(ratio, objects)
#print("train idx ",i[0])
#print("test idx ",i[1])
print("train obj:")
tr=[]
for j in i[0]:
  #print(objects[int(j)])
  tr.append(objects[int(j)])
tr_u=set(tr)

print("test obj:")
te=[]
for j in i[1]:
  #print(objects[int(j)])
  te.append(objects[int(j)])
te_u=set(te)

print(tr_u,len(tr))
print(te_u,len(te))
splits=i

In [6]:
tfms  = [None, [Categorize()]]

dsets_arousal = TSDatasets(X, Y_arousal, tfms=tfms, splits=splits, inplace=True)
dsets_valence = TSDatasets(X, Y_valence, tfms=tfms, splits=splits, inplace=True)
dsets_stress = TSDatasets(X, Y_stress, tfms=tfms, splits=splits, inplace=True)

bs=128
dls_stress = TSDataLoaders.from_dsets(dsets_stress.train, dsets_stress.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_arousal = TSDataLoaders.from_dsets(dsets_arousal.train, dsets_arousal.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)
dls_valence = TSDataLoaders.from_dsets(dsets_valence.train, dsets_valence.valid, bs=bs, batch_tfms=[TSStandardize()], num_workers=0)

model_stress = InceptionTime(dls_stress.vars, dls_stress.c)
model_arousal = InceptionTime(dls_arousal.vars, dls_arousal.c)
model_valence = InceptionTime(dls_valence.vars, dls_valence.c)

learn_stress = Learner(dls_stress, model_stress, metrics=accuracy)
learn_arousal = Learner(dls_arousal, model_arousal, metrics=accuracy)
learn_valence = Learner(dls_valence, model_valence, metrics=accuracy)

learn_stress.save('S0')
learn_arousal.save('S0')
learn_valence.save('S0')

Path('models/S0.pth')

In [7]:
learn_stress.load('S0')
learn_stress.lr_find()

SuggestedLRs(valley=0.0010000000474974513)

In [ ]:
learn_stress.fit_one_cycle(100,lr_max=1e-3)
learn_stress.save('S1')